## Manage Dataproc Workflows using gcloud Commands
Let us see how we can manage Dataproc Workflows using gcloud commands.
* Step 1: Create Dataproc Workflow
* Step 2: Configure active Dataproc cluster (we can also configure new cluster)
* Step 3: Add Spark SQL or Pyspark Jobs to Dataproc Workflow with Dependencies
* Step 4: Run and Validate the Dataproc Workflow

We can take care of all the steps using `gcloud` commands.

In [1]:
!gcloud config set dataproc/region us-central1

Updated property [dataproc/region].


Here is the command to delete Dataproc Workflow (multiline approach doesn't work on Windows)

```shell
gcloud dataproc workflow-templates \
    delete wf-daily-product-revenue
```

In [3]:
!gcloud dataproc workflow-templates delete wf-daily-product-revenue --quiet

Here is the command to create Dataproc Workflow.

```shell
gcloud dataproc workflow-templates \
    create wf-daily-product-revenue
```

In [4]:
!gcloud dataproc workflow-templates create wf-daily-product-revenue

Here is the command to attach running or active Dataproc Cluster to the Dataproc Workflow. We need to specify the label for the cluster.

```shell
gcloud dataproc workflow-templates \
    set-cluster-selector \
    wf-daily-product-revenue \
    --cluster-labels goog-dataproc-cluster-name=aidataprocdev
```

In [5]:
!gcloud dataproc workflow-templates set-cluster-selector wf-daily-product-revenue --cluster-labels goog-dataproc-cluster-name=aidataprocdev

Here are the commands to add Spark SQL Jobs to the Dataproc Workflow.

```shell
gcloud dataproc workflow-templates add-job \
    spark-sql --step-id=job-cleanup \
    --file=gs://airetail/scripts/daily_product_revenue/cleanup.sql \
    --workflow-template=wf-daily-product-revenue

# File Format Converted jobs with dependency on cleanup
gcloud dataproc workflow-templates add-job \
    spark-sql --step-id=job-convert-orders \
    --file=gs://airetail/scripts/daily_product_revenue/file_format_converter.sql \
    --params=bucket_name=gs://airetail,table_name=orders \
    --workflow-template=wf-daily-product-revenue \
    --start-after=job-cleanup

gcloud dataproc workflow-templates add-job \
    spark-sql --step-id=job-convert-order-items \
    --file=gs://airetail/scripts/daily_product_revenue/file_format_converter.sql \
    --params=bucket_name=gs://airetail,table_name=order_items \
    --workflow-template=wf-daily-product-revenue \
    --start-after=job-cleanup

# Last Job which depends on convert orders and order_items jobs
gcloud dataproc workflow-templates add-job \
    spark-sql --step-id=job-daily-product-revenue \
    --file=gs://airetail/scripts/daily_product_revenue/compute_daily_product_revenue.sql \
    --params=bucket_name=gs://airetail \
    --workflow-template=wf-daily-product-revenue \
    --start-after=job-convert-orders,job-convert-order-items
```

In [6]:
!gcloud dataproc workflow-templates add-job spark-sql --step-id=job-cleanup --file=gs://airetail/scripts/daily_product_revenue/cleanup.sql --workflow-template=wf-daily-product-revenue

createTime: '2022-10-07T15:51:51.085329Z'
id: wf-daily-product-revenue
jobs:
- sparkSqlJob:
    queryFileUri: gs://airetail/scripts/daily_product_revenue/cleanup.sql
  stepId: job-cleanup
name: projects/tidy-fort-361710/regions/us-central1/workflowTemplates/wf-daily-product-revenue
placement:
  clusterSelector:
    clusterLabels:
      goog-dataproc-cluster-name: aidataprocdev
updateTime: '2022-10-07T15:52:03.543638Z'
version: 3


In [7]:

!gcloud dataproc workflow-templates add-job spark-sql --step-id=job-convert-orders --file=gs://airetail/scripts/daily_product_revenue/file_format_converter.sql --params=bucket_name=gs://airetail,table_name=orders --workflow-template=wf-daily-product-revenue --start-after=job-cleanup

createTime: '2022-10-07T15:51:51.085329Z'
id: wf-daily-product-revenue
jobs:
- sparkSqlJob:
    queryFileUri: gs://airetail/scripts/daily_product_revenue/cleanup.sql
  stepId: job-cleanup
- prerequisiteStepIds:
  - job-cleanup
  sparkSqlJob:
    queryFileUri: gs://airetail/scripts/daily_product_revenue/file_format_converter.sql
    scriptVariables:
      bucket_name: gs://airetail
      table_name: orders
  stepId: job-convert-orders
name: projects/tidy-fort-361710/regions/us-central1/workflowTemplates/wf-daily-product-revenue
placement:
  clusterSelector:
    clusterLabels:
      goog-dataproc-cluster-name: aidataprocdev
updateTime: '2022-10-07T15:52:12.748643Z'
version: 4


In [8]:
!gcloud dataproc workflow-templates add-job spark-sql --step-id=job-convert-order-items --file=gs://airetail/scripts/daily_product_revenue/file_format_converter.sql --params=bucket_name=gs://airetail,table_name=order_items --workflow-template=wf-daily-product-revenue --start-after=job-cleanup

createTime: '2022-10-07T15:51:51.085329Z'
id: wf-daily-product-revenue
jobs:
- sparkSqlJob:
    queryFileUri: gs://airetail/scripts/daily_product_revenue/cleanup.sql
  stepId: job-cleanup
- prerequisiteStepIds:
  - job-cleanup
  sparkSqlJob:
    queryFileUri: gs://airetail/scripts/daily_product_revenue/file_format_converter.sql
    scriptVariables:
      bucket_name: gs://airetail
      table_name: orders
  stepId: job-convert-orders
- prerequisiteStepIds:
  - job-cleanup
  sparkSqlJob:
    queryFileUri: gs://airetail/scripts/daily_product_revenue/file_format_converter.sql
    scriptVariables:
      bucket_name: gs://airetail
      table_name: order_items
  stepId: job-convert-order-items
name: projects/tidy-fort-361710/regions/us-central1/workflowTemplates/wf-daily-product-revenue
placement:
  clusterSelector:
    clusterLabels:
      goog-dataproc-cluster-name: aidataprocdev
updateTime: '2022-10-07T15:52:18.400868Z'
version: 5


In [9]:
!gcloud dataproc workflow-templates add-job spark-sql --step-id=job-daily-product-revenue --file=gs://airetail/scripts/daily_product_revenue/compute_daily_product_revenue.sql --params=bucket_name=gs://airetail --workflow-template=wf-daily-product-revenue --start-after=job-convert-orders,job-convert-order-items

createTime: '2022-10-07T15:51:51.085329Z'
id: wf-daily-product-revenue
jobs:
- sparkSqlJob:
    queryFileUri: gs://airetail/scripts/daily_product_revenue/cleanup.sql
  stepId: job-cleanup
- prerequisiteStepIds:
  - job-cleanup
  sparkSqlJob:
    queryFileUri: gs://airetail/scripts/daily_product_revenue/file_format_converter.sql
    scriptVariables:
      bucket_name: gs://airetail
      table_name: orders
  stepId: job-convert-orders
- prerequisiteStepIds:
  - job-cleanup
  sparkSqlJob:
    queryFileUri: gs://airetail/scripts/daily_product_revenue/file_format_converter.sql
    scriptVariables:
      bucket_name: gs://airetail
      table_name: order_items
  stepId: job-convert-order-items
- prerequisiteStepIds:
  - job-convert-orders
  - job-convert-order-items
  sparkSqlJob:
    queryFileUri: gs://airetail/scripts/daily_product_revenue/compute_daily_product_revenue.sql
    scriptVariables:
      bucket_name: gs://airetail
  stepId: job-daily-product-revenue
name: projects/tidy-fort-36

Here is the command to instantiate or run Dataproc Workflow.

```shell
gcloud dataproc workflow-templates \
    instantiate wf-daily-product-revenue
```

In [10]:
# This will take some time to run

!gcloud dataproc workflow-templates instantiate wf-daily-product-revenue

Waiting on operation [projects/tidy-fort-361710/regions/us-central1/operations/70558a19-e99d-3b5e-aeee-021a2c96d997].
WorkflowTemplate [wf-daily-product-revenue] RUNNING
Job ID job-cleanup-vllxvh7adqibo RUNNING
Job ID job-cleanup-vllxvh7adqibo COMPLETED
Job ID job-convert-orders-vllxvh7adqibo RUNNING
Job ID job-convert-order-items-vllxvh7adqibo RUNNING
Job ID job-convert-orders-vllxvh7adqibo COMPLETED
Job ID job-convert-order-items-vllxvh7adqibo COMPLETED
Job ID job-daily-product-revenue-vllxvh7adqibo RUNNING
WorkflowTemplate [wf-daily-product-revenue] DONE
Job ID job-daily-product-revenue-vllxvh7adqibo COMPLETED
